### Import Dependencies

In [3]:
import sys
sys.path.append("../")
import pandas as pd

from utils.prompts import render
from utils.llm_client import LLMClient
from utils.logging_utils import log_llm_call
from utils.router import pick_model,should_use_reasoning_model
from IPython.display import display, Markdown

### Loading data from sample.txt file

In [4]:
file_path = '../data/raw/Sample Messages.txt'

with open(file_path, 'r', encoding='utf-8') as f:
    messages = [line.strip() for line in f if line.strip()]

print(f"Loaded {len(messages)} messages\n")

Loaded 50 messages



### Adding few-shot examples

In [5]:
few_shot_examples = """Examples:
1. Message: "SOS: 5 people trapped on a roof in Ja-Ela (Gampaha). Need rescue boat."
   Output: District: Gampaha | Intent: Rescue | Priority: High

2. Message: "Donation drive starting at Town Hall. We need water bottles and biscuits."
   Output: District: Colombo | Intent: Supply | Priority: Low

3. Message: "Update: Kandy road cleared near Peradeniya. Traffic moving slowly."
   Output: District: Kandy | Intent: Info | Priority: Low

4. Message: "Warning: Dengue risk rising in Gampaha due to stagnant water."
   Output: District: Gampaha | Intent: Other | Priority: Low

5. Message: "BREAKING: Water levels in Kelani River (Colombo) have reached 9.5 meters."
   Output: District: Colombo | Intent: Info | Priority: High
"""

#### Refactor the classification prompt using skeleton.v1.

In [6]:
prompt_text, spec = render(
    "skeleton.v1",
    role="You are a disaster response classifier for Sri Lankan crisis messages",
    task="Classify the incoming message by extracting the district, determining the intent, and assigning priority",
    context=few_shot_examples,
    constraints="""- Use ONLY these categories: Rescue, Supply, Info, Other
                - District must be a valid Sri Lankan district or [None]
                - Priority is High for life-threatening situations (SOS, trapped, injured), Low otherwise
                - Do NOT confuse news reports with actual SOS calls""",
    format="District: [Name] | Intent: [Category] | Priority: [High/Low]",
    checks="""- Output must be EXACTLY one line following the format
                - No extra text, explanations, or multiple outputs
                - If district is unclear, use [None]"""
)

### Iterate through the loaded messages, classify each one using your skeleton.v1, and store the results. 

In [7]:
model = pick_model('groq', 'general')
client = LLMClient('groq', model)

classifications=[]

for msg in messages:
    full_prompt = f"{prompt_text}\n\nNow classify this message:\nMessage: \"{msg}\""
    
    msg_payload = [{'role': 'user', 'content': full_prompt}]
    
    # Get classification
    response = client.chat(msg_payload, temperature=0.0)
    classification = response['text'].strip()
    classifications.append(classification)
    

### Save the above classified results into a CSV/Excel file using pandas named output/classified_messages.xlsx 

In [8]:
data=[]
for d in classifications:
    d = d.split('|')
    result={
        'District':d[0].split(':')[1].strip(),
        'Intent':d[1].split(':')[1].strip(),
        'Priority':d[2].split(':')[1].strip()
    }
    data.append(result)

df = pd.DataFrame(data)
df.to_excel('../data/output/classified_messages.xlsx',index=False)